In [1]:
from data_sampler import WeightedSampler
from dataset import CARLADataset
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch
import os

In [2]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")


# test commit
config = {"used_inputs": ["rgb", "depth", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle"],
        "seq_len": 2
        }

def transform(sample):
        """
        This function reshapes every input in a sample to be
        shape = [seq_len, data_shape]
        """
        for key in sample:
                data_tensor = torch.Tensor(sample[key])
                shape = list(data_tensor.shape)
                shape.remove(config["seq_len"])
                shape_new = [config["seq_len"]] + shape
                sample[key] = data_tensor.reshape(shape_new)
        return sample

# data_transform = transforms.Compose([
#         torch.stack()
#         # transforms.ToTensor(),
#         # torch.reshape(config["seq_len"], ...)
#         # transforms.Normalize(mean=[0.485, 0.456, 0.406],
#         #                      std=[0.229, 0.224, 0.225])
# ])

dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=None)
weighted_sampler = WeightedSampler(dataset=dataset)
print(dataset.__len__())

217


In [3]:
dataset.data_shapes

[[160, 960, 3], [160, 960, 3], None]

In [4]:
dataset.df_meta_data.columns[2] == "measurementsa"

False

In [5]:
dataset.df_meta_data

,route,depth,rgb,measurements
0,Town10HD_Scenario10_route16_11_28_18_26_19,0000.png,0000.png,0000.json
1,Town10HD_Scenario10_route16_11_28_18_26_19,0001.png,0001.png,0001.json
2,Town10HD_Scenario10_route16_11_28_18_26_19,0002.png,0002.png,0002.json
3,Town10HD_Scenario10_route16_11_28_18_26_19,0003.png,0003.png,0003.json
4,Town10HD_Scenario10_route16_11_28_18_26_19,0004.png,0004.png,0004.json
...,...,...,...,...
212,Town10HD_Scenario10_route3_11_28_17_28_17,0148.png,0148.png,0148.json
213,Town10HD_Scenario10_route3_11_28_17_28_17,0149.png,0149.png,0149.json
214,Town10HD_Scenario10_route3_11_28_17_28_17,0150.png,0150.png,0150.json
215,Town10HD_Scenario10_route3_11_28_17_28_17,0151.png,0151.png,0151.json


In [6]:
dl = DataLoader(dataset=dataset, batch_size=16, num_workers=0, sampler=weighted_sampler, collate_fn=None)

In [7]:
# Weird: When num_workers increases (>0) then it takes longer to loop over batches??
count = 0
for batch in dl:
    count += 1
print(count)

---------HHELLLO------------


IndexError: index 2 is out of bounds for axis 1 with size 2

In [16]:
for i in range(10):
    if i == 5:
        print("ksdfk")
        continue
    print(i)
    print("bla")

0
bla
1
bla
2
bla
3
bla
4
bla
ksdfk
6
bla
7
bla
8
bla
9
bla


In [18]:
for i in range(10):
    print(i)
    if i == 5:
        print("ksdfk")
        continue
    for a in range(2):
        print(a)
    

0
0
1
1
0
1
2
0
1
3
0
1
4
0
1
5
ksdfk
6
0
1
7
0
1
8
0
1
9
0
1
